## Import Libraries

In [30]:
!pip install ipywidgets
!pip install sentence-transformers

/bin/bash: /home/teguh/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/teguh/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [31]:
from tqdm import tqdm as tqdm
from datasets import load_dataset
dataset = load_dataset("LazarusNLP/stsb_mt_id")

Found cached dataset parquet (/home/teguh/.cache/huggingface/datasets/LazarusNLP___parquet/LazarusNLP--stsb_mt_id-53495c8bc04ac9ed/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
try:
    dataset['train'] = dataset.pop('validation')
except:
    pass

try:
    dataset['validation'] = dataset.pop('test')
except:
    pass


In [33]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['domain', 'data', 'type', 'score', 'correlation', 'text_1', 'text_2'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['domain', 'data', 'type', 'score', 'correlation', 'text_1', 'text_2'],
        num_rows: 1379
    })
})


## Normalize the correlation score

In [34]:
train_cor = [cor['correlation'] for cor in dataset['train']]
val_cor = [cor['correlation'] for cor in dataset['validation']]

norm_train_cor = [float(i)/5.0 for i in train_cor]
norm_val_cor = [float(i)/5.0 for i in val_cor]

## Implement SBERT

In [35]:
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('indobenchmark/indobert-base-p2', max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
sts_bert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])